In [ ]:
# necessary to use pyaudio
!wget https://files.portaudio.com/archives/pa_stable_v190700_20210406.tgz
!tar -xvzf pa_stable_v190700_20210406.tgz

!portaudio/configure
!make
!make install

# you also need to run
# sudo apt install python3-all-dev
# sudo apt install portaudio19-dev

--2025-02-22 17:23:08--  https://files.portaudio.com/archives/pa_stable_v190700_20210406.tgz
Resolving files.portaudio.com (files.portaudio.com)... 44.205.198.66
Connecting to files.portaudio.com (files.portaudio.com)|44.205.198.66|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1462695 (1.4M) [application/x-gzip]
Saving to: ‘pa_stable_v190700_20210406.tgz’

pa_stable_v190700_2 100%[===================>]   1.39M   782KB/s    in 1.8s    

2025-02-22 17:23:10 (782 KB/s) - ‘pa_stable_v190700_20210406.tgz’ saved [1462695/1462695]

portaudio/
portaudio/.editorconfig
portaudio/.gitattributes
portaudio/.github/
portaudio/.github/ISSUE_TEMPLATE/
portaudio/.github/ISSUE_TEMPLATE/bug_report.md
portaudio/.github/workflows/
portaudio/.github/workflows/MSBuild.yml
portaudio/.github/workflows/c-cpp.yml
portaudio/.gitignore
portaudio/CMakeLists.txt
portaudio/Doxyfile
portaudio/Doxyfile.developer
portaudio/LICENSE.txt
portaudio/Makefile.in
portaudio/README.configure.txt
porta

In [5]:
import speech_recognition as sr
from gtts import gTTS
import os
import playsound
import wikipedia
import webbrowser

In [6]:
def speak(text):
    tts = gTTS(text=text, lang='en')
    filename = "voice.mp3"
    try:
        os.remove(filename)
    except OSError:
        pass
    tts.save(filename)
    playsound.playsound(filename)

In [7]:
print(sr.Microphone.list_microphone_names())

[]


ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5204:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5204:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1342:(snd_func_refer) error evaluating name
ALSA lib conf.c:5204:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5727:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2721:(snd_pcm_open_noupdate) Unknown PCM sysdefault
ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5204:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5204:(_snd_config_evaluate) function snd_func_concat returned error: No

In [15]:
def get_audio():
    r = sr.Recognizer()
    with sr.Microphone() as source:
        r.pause_threshold = 1
        # wait for a second to let the recognizer adjust the
        # energy threshold based on the surrounding noise level
        r.adjust_for_ambient_noise(source, duration=1)
        audio = r.listen(source)
        said = ""
        try:
            said = r.recognize_google(audio)
            print(said)
        except sr.UnknownValueError:
            speak("Sorry, I did not get that.")
        except sr.RequestError:
            speak("Sorry, the service is not available")
    return said.lower()

In [19]:
import requests
from geopy.geocoders import Nominatim

def get_user_location():
    try:
        response = requests.get("https://ipinfo.io/json")
        data = response.json()
        lat, lon = map(float, data["loc"].split(","))
        return lat, lon
    except Exception as e:
        print("Error fetching location:", e)
        return None, None

def get_nearest_drugstore():
    lat, lon = get_user_location()
    if lat is None or lon is None:
        return "Could not determine your location."

    geolocator = Nominatim(user_agent="my-unique-user-agent-for-drugstore-app")
    location = geolocator.reverse((lat, lon), exactly_one=True)

    if location and "address" in location.raw:
        place = location.raw["address"]
        return f"Nearest known location is in {place.get('road')}, {place.get('house_number')}, {place.get('city')}."
    return "Could not find a drugstore nearby."


In [20]:
def respond(text):
    print("Text from get audio " + text)
    if 'youtube' in text:
        speak("What do you want to search for?")
        keyword = get_audio()
        if keyword!= '':
            url = f"https://www.youtube.com/results?search_query={keyword}"
            webbrowser.get().open(url)
            speak(f"Here is what I have found for {keyword} on youtube")
    elif 'search' in text:
        speak("What do you want to search for?")
        query = get_audio()
        if query !='':
            result = wikipedia.summary(query, sentences=3)
            speak("According to wikipedia")
            print(result)
            speak(result)
    elif 'drugstore' in text:
        location = get_nearest_drugstore()
        speak(location)
    elif 'exit' in text:
        speak("Goodbye, till next time")
        exit()

while True:
    print("I am listening...")
    text = get_audio()
    respond(text)

I am listening...


OSError: No Default Input Device Available